<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers
!pip install evaluate numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-fxkjneki
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-fxkjneki
  Resolved https://github.com/huggingface/accelerate to commit 5b3f3b99d6aa1d71eaa3380af87a6ca6126505fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.25.0.dev0-py3-no

# Load the dataset

In [ ]:
from datasets import load_dataset

dataset_name = "covid_qa_deepset"
dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(train_size=1413,test_size=303)

dataset = dataset.filter(lambda example: not "answer_category" in example['answers'])

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/303 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 303
    })
})

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AutoTokenizer

# load distilled bert tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
#tokenizer.pad_token = tokenizer.eos_token


In [ ]:
tokenizer("I am Son")

{'input_ids': [101, 1045, 2572, 2365, 102], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:

def preprocess_training(example):
    inputs = tokenizer(
        example['question'],
        example["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = example["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_eval(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

training_dataset = dataset['train']
training_dataset = training_dataset.map(preprocess_training, batched=True, remove_columns=dataset['train'].column_names)

eval_dataset = dataset['test']
eval_dataset = eval_dataset.map(preprocess_training, batched=True, remove_columns=dataset['test'].column_names)


Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

In [75]:
test_dataset = load_dataset(dataset_name, split="train[1969:]")

In [ ]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1413
})

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 303
})

In [76]:
test_dataset

Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
    num_rows: 50
})

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# Load the model

In [ ]:
import torch
import transformers
from transformers import AutoModelForQuestionAnswering, BitsAndBytesConfig, BertForQuestionAnswering, AutoConfig

model_name = "distilbert-base-uncased-distilled-squad"


model = DistilBertForQuestionAnswering.from_pretrained(
    model_name,
)
model.config.use_cache = False


In [ ]:
#Reuse the finetuned model uploaded on Huggingface
model = DistilBertForQuestionAnswering.from_pretrained(
    "goodcoffee/Distilled_bert_CovidQA_model"
)
model.config.use_cache = False

# Load evaluation metrics

In [42]:
import evaluate
import numpy as np

metric = evaluate.load("f1")
#metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions[0], references=labels[0], average="macro")

In [43]:
metric.description

'\nThe F1 score is the harmonic mean of the precision and recall. It can be computed with the equation:\nF1 = 2 * (precision * recall) / (precision + recall)\n'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="Distilled_bert_CovidQA_model",
    evaluation_strategy = "steps",
    save_steps = 10,
    logging_steps = 10,
    max_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


In [46]:
from transformers import Trainer

max_seq_length = 2048

trainer = Trainer(
    model=model,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 50
})

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key="2ba90e109c0da4331467290174e54e3483f0d494")
wandb.init(project="QLoRA covidQA")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobbyngoson204 (intelligentsystem). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy
10,1.126500,0.906048,0.801980
20,0.609300,0.718573,0.841584
30,0.658100,0.718396,0.838284
40,0.871600,0.726818,0.828383
50,0.819900,0.686123,0.864686
60,0.920200,0.678616,0.851485
70,0.664600,0.650183,0.851485
80,0.549400,0.663274,0.848185
90,0.763000,0.688111,0.844884
100,0.509100,0.664566,0.854785


TrainOutput(global_step=500, training_loss=0.38069887399673463, metrics={'train_runtime': 9669.35, 'train_samples_per_second': 0.827, 'train_steps_per_second': 0.052, 'total_flos': 1038038879754240.0, 'train_loss': 0.38069887399673463, 'epoch': 5.62})

#Run evaluation

In [47]:
trainer.evaluate()

{'eval_loss': 0.8438284397125244,
 'eval_f1': 0.294705621956618,
 'eval_runtime': 72.1499,
 'eval_samples_per_second': 4.2,
 'eval_steps_per_second': 0.263}

In [79]:
from evaluate import evaluator

task_evaluator = evaluator("question-answering")
results = task_evaluator.compute(
    model_or_pipeline="goodcoffee/Distilled_bert_CovidQA_model",
    data=test_dataset,
    metric="squad",
)

print(results)

Filter:   0%|          | 0/50 [00:00<?, ? examples/s]

ValueError: ignored

In [77]:

from transformers import QuestionAnsweringPipeline

pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

total_f1 = 0
total_accuracy = 0

for ins in test_dataset:
  ans = pipeline(question=ins['question'], context=ins['context'], max_answer_len=50, max_question_len=300)
  ref_tokens = tokenizer(" " + ins["answers"]["text"][0])["input_ids"]
  ans_tokens = tokenizer(ans["answer"])["input_ids"]
  common_tokens = set(ans_tokens) & set(ref_tokens)
  precision = len(common_tokens) / len(ans_tokens)
  recall = len(common_tokens) / len(ref_tokens)
  total_accuracy += precision
  print(tokenizer.decode(ans_tokens), "|", tokenizer.decode(ref_tokens), "|")
  if (len(common_tokens) == 0):
    total_f1 += 0
    print(0)
  else:
    f1 = 2 * precision * recall / (precision + recall)
    total_f1 += f1
    print(f1)

[CLS] january 31, 2020 [SEP] | [CLS] january 30, 2020 [SEP] |
0.8333333333333334
[CLS] january 31, 2020 [SEP] | [CLS] january 31, 2020, [SEP] |
0.923076923076923
[CLS] fever and respiratory symptoms [SEP] | [CLS] fever and cough [SEP] |
0.7272727272727272
[CLS] fever and respiratory symptoms [SEP] | [CLS] patients with underlying medical conditions and the elderly [SEP] |
0.37499999999999994
[CLS] healthcare personnel [SEP] | [CLS] patients with underlying medical conditions and the elderly [SEP] |
0.28571428571428575
[CLS] 2 and 14 days [SEP] | [CLS] between 2 and 14 days [SEP] |
0.923076923076923
[CLS] through contact with infected respiratory secretions [SEP] | [CLS] close contact with an infected person [SEP] |
0.5882352941176471
[CLS] 34 the possibility that patients may be infectious prior to symptom onset further compounds the difficulty of containing the virus and effectively preventing transmission. [SEP] | [CLS] although the likelihood of coinfection of 2019 - ncov and anothe

In [78]:
print("F1 average score:", total_f1 / 50)
print("Accuracy average score: ", total_accuracy / 50)

F1 average score: 0.633776704091313
Accuracy average score:  0.7655031439244173


# Inference

In [52]:
# Before training
from transformers import pipeline, QuestionAnsweringPipeline

question = "Where did Covid-19 originated"
context = 'According to epidemiological studies, the Huanan Market in Wuhan was the early and main epicentre of SARS‐CoV‐2 infection'
print(question)

pipe = pipeline(model="distilbert-base-uncased-distilled-squad")
pipe(question=question, context=context, max_answer_len=300, max_question_len=300)

Where did Covid-19 originated


{'score': 0.35626348853111267,
 'start': 42,
 'end': 64,
 'answer': 'Huanan Market in Wuhan'}

In [53]:
#After training
from transformers import QuestionAnsweringPipeline
question = "Where did Covid-19 originated"
context = "According to epidemiological studies, the Huanan Market in Wuhan was the early and main epicentre of SARS‐CoV‐2 infection"
print(question)
#print(context)

pipe2 = QuestionAnsweringPipeline(model=model, tokenizer = tokenizer)
pipe2(question=question, context=context, max_answer_len=300, max_question_len=300)

Where did Covid-19 originated


{'score': 0.4766525328159332,
 'start': 42,
 'end': 64,
 'answer': 'Huanan Market in Wuhan'}